#IMPORTS

In [545]:
#hindi wordnet and codecs,re
import codecs
import re
!pip install pyiwn
import pyiwn
iwn = pyiwn.IndoWordNet()

#sklearn,numpy,pandas,pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

#models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as rfc

#accuracy metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix,classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#PREPROCESSING

In [81]:
class Tokenizer():
  def __init__(self,text=None):
    if text is  not None:
      self.text=text
      self.cleanText()
    else:
      self.text=None

    self.sentences=[]
    self.tokens=[]
    self.stemmed_word=[]
    self.final_list=[]

  def gen_sentences(self):
    text=self.text
    self.sentences=text.split("। ")
    #print(self.sentences)

  def printTokens(self):
    for i in self.tokens:
      print(i)

  def spaceTok(self,sentences_list):
    tokens=[]
    for each in sentences_list:
        word_list=each.split(' ')
        word_list[-1] = word_list[-1].replace("।",'')
        tokens=tokens+word_list

    return tokens

  def cleanText(self):
    text=self.text
    text=text.strip()
    text=re.sub(r'(\d+)',r'',text)
    text=text.replace(',','')
    text=text.replace('"','')
    text=text.replace('(','')
    text=text.replace(')','')
    text=text.replace('"','')
    text=text.replace(':','')
    text=text.replace("'",'')
    text=text.replace("‘‘",'')
    text=text.replace("’’",'')
    text=text.replace("''",'')
    text=text.replace(".",'')
    text=text.replace("-",' ')
    text=text.replace("  ",' ')
    self.text=text

  def removeStopWords(self,token_list):
    f=codecs.open("stopwords.txt",encoding='utf-8')
    stopwords=[x.strip() for x in f.readlines()]
    #print(stopwords)
    #print(token_list)
    tokens=[i for i in token_list if i not in stopwords]
    self.final_tokens=tokens
    return tokens

  def stemWords(self,word):
    suffixes = {
    1: ["ो","े","ू","ु","ी","ि","ा"],
    2: ["कर","ाओ","िए","ाई","ाए","ने","नी","ना","ते","ीं","ती","ता","ाँ","ां","ों","ें"],
    3: ["ाकर","ाइए","ाईं","ाया","ेगी","ेगा","ोगी","ोगे","ाने","ाना","ाते","ाती","ाता","तीं","ाओं","ाएं","ुओं","ुएं","ुआं"],
    4: ["ाएगी","ाएगा","ाओगी","ाओगे","एंगी","ेंगी","एंगे","ेंगे","ूंगी","ूंगा","ातीं","नाओं","नाएं","ताओं","ताएं","ियाँ","ियों","ियां"],
    5: ["ाएंगी","ाएंगे","ाऊंगी","ाऊंगा","ाइयाँ","ाइयों","ाइयां"],
    }
    for L in 5, 4, 3, 2, 1:
      if len(word) > L + 1:
        for suf in suffixes[L]:
          #print type(suf),type(word),word,suf
          if word.endswith(suf):
            #print 'h'
            return word[:-L]
    return word

  def scriptValidation(self,token_list):
    tokens = []
    for tok in token_list:
      #print(i)
      for ch in tok:
        #print(x)
        if ord(ch) not in list(range(2304,2432)):
          tok = tok.replace(ch,'')
      if tok!="":
        tokens.append(tok)

    return tokens

  def tokenize(self):
    if not self.sentences:
      self.gen_sentences()
      sentences_list=self.sentences
      #print(sentences_list)
      token_list = self.spaceTok(sentences_list)
      #print(token_list)
      token_list = self.scriptValidation(token_list)
      #print(token_list)
      token_list = self.removeStopWords(token_list)
      #print(token_list)
      tokens = []
      for tok in token_list:
        tokens.append(self.stemWords(tok))
      self.tokens=tokens
      return tokens

#READ DATASET

In [500]:
df = pd.read_csv("Train.csv")

In [501]:
df.head(10)

,id,SENTENCE1,SENTENCE2,label
0,1,ब्रैड क्रेलिन ने ऑस्ट्रेलिया के दौरे पर 6 अन्य...,ब्रैड क्रेलिन ने ऑस्ट्रेलिया के दौरे पर ऑस्ट्र...,0
1,2,वे हमें आनंद देने के लिए वहाँ थे और वे हमारे ल...,वे हमारे लिए आनंद लेने के लिए वहाँ थे और वे प्...,1
2,3,"जून 1902 में युद्ध की समाप्ति के बाद, हिगिंस न...",अगस्त 1902 में युद्ध की समाप्ति के बाद अगस्त म...,1
3,4,फोर रिवर्स काउंसिल और ऑडबोन काउंसिल के विलय से...,शॉनी ट्रेल्स काउंसिल का गठन फोर रिवर्स काउंसिल...,1
4,5,समूह ने बड़े पैमाने पर दौरा किया और इज़राइल मे...,समूह ने बड़े पैमाने पर दौरा किया और इज़राइल मे...,1
5,6,कैथी और उनके पति पीट बीले (पीटर डीन) आर्थिक रू...,कैथी और उनके पति पीटर डीन (पीट बीले) आर्थिक रू...,1
6,7,तिमोरा डायहरोडा नोक्टुइडे परिवार की पतंग की एक...,दिहरोदा नोक्टुइडे परिवार का एक प्रकार का कीट ह...,1
7,8,इनमें से अधिकांश होटल ओक स्ट्रीट से पाइन स्ट्र...,इनमें से अधिकांश घर ओक स्ट्रीट से पाइन स्ट्रीट...,0
8,9,"जो आर। कैम्पा जूनियर, संयुक्त राज्य अमेरिका की...",जो आर। कैम्पा जूनियर एक पूर्व अमेरिकी नेवी मैट...,1
9,10,अपवाद 2005 के अंत और 2009 के बीच था जब उन्होंन...,"2005 के अंत और 2009 के बीच अपवाद था, जब वह स्व...",1


In [502]:
df.shape

(15625, 4)

In [503]:
df.set_index("id", inplace = True)

In [504]:
df.shape

(15625, 3)

In [505]:
df = df.sample(frac=0.4)

In [506]:
df = df.reset_index(drop=True)

In [507]:
df.shape

(6250, 3)

In [508]:
label = df['label']

In [509]:
print(df.iloc[0])

SENTENCE1    Farrer का जन्म लंदन में हुआ था, जो कलाकार Thom...
SENTENCE2    थॉमस चार्ल्स फ़ेरर का जन्म लंदन में हुआ था, जो...
label                                                        0
Name: 0, dtype: object


#COSINE SIMILARITY

In [510]:
def cosine_similarity(token_list1,token_list2):
  l1 =[]
  l2 =[]
  token_list1 = set(token_list1)
  token_list2 = set(token_list2)
  # form a set containing keywords of both strings 
  rvector = token_list1.union(token_list2)
  
  #print(rv)
  for w in rvector:
      if w in token_list1: l1.append(1) # create a vector
      else: l1.append(0)
      
      if w in token_list2: l2.append(1)
      else: l2.append(0)

  #print(l1,l2)
  c = 0
    
  # cosine formula 
  for i in range(len(rvector)):
          c+= l1[i]*l2[i] 

  #print(c)

  try:
    cosine = round(c / float((sum(l1)**0.5)*(sum(l2)**0.5)),2)    
  except:
    return 0
  else:
    if cosine >= 0 and cosine <= 0.33:
      return 0
    elif cosine >= 0.34 and cosine <= 0.67:
      return 1
    elif cosine >= 0.68:
      return 2
    #print("similarity: ", cosine)

#Jaccard similarity

In [511]:
def jaccard_similarity(x,y):
  x = set(x)
  y = set(y)
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(x.intersection(y))
  union_cardinality = len(x.union(y))

  try:
    jaccard = round(intersection_cardinality/union_cardinality,2)
  except:
    return 0
  else:
    if jaccard >= 0 and jaccard <= 0.33:
      return 0
    elif jaccard >= 0.34 and jaccard <= 0.67:
      return 1
    elif jaccard >= 0.68:
      return 2

#Synonym–level matching

In [512]:
def syno_match(token_list1,token_list2):
  count = 0
  for each in token_list1:
    #print("each: ",each)
    if each in token_list2:
      #print("count++: ",each)
      count+=1
    else:
      try:
        word = iwn.synsets(each)
      except:
        #print("error")
        pass
      else:
        syn_words = []
        for i in range(0,len(word)):
          syn_words.append(word[i].head_word())
        #print(syn_words)
        for w in syn_words:
          if w in token_list2:
            count+=1
            #print("count++: ",each)
  try:
  #print(count)
    sim = round(count/(max(len(token_list1),len(token_list2))),2)
  except:
    return 0
  else:
    if sim >= 0 and sim <= 0.33:
      return 0
    elif sim >= 0.34 and sim <= 0.67:
      return 1
    elif sim >= 0.68:
      return 2
    #print(sim)

#BIGRAM


In [513]:
def bigram_similarity(token_list1,token_list2):
  bigram_list1,bigram_list2 = [],[]
  for i in range(0,len(token_list1)-1):
    bigram_list1.append([token_list1[i],token_list1[i+1]])

  for i in range(0,len(token_list2)-1):
    bigram_list2.append([token_list2[i],token_list2[i+1]])

  nt1 = map(tuple, bigram_list1)
  nt2 = map(tuple, bigram_list2)

  st1 = set(nt1)
  st2 = set(nt2)

  rvector = st1.intersection(st2)
  #print(rvector)
  try:
    bigram = round(len(rvector)/(len(bigram_list1)+len(bigram_list2)),2)
  except:
    return 0
  else:
    if bigram >= 0 and bigram <= 0.33:
      return 0
    elif bigram >= 0.34 and bigram <= 0.67:
      return 1
    elif bigram >= 0.68:
      return 2

#FEATURE EXTRACTION

In [514]:
def features_extraction(df):
  corpus = []
  cosine_data,jaccard_data,bigram_data,syno_data = [],[],[],[]
  for i in range(0,len(df)):
    #print("iter: ",i)
    #print(df['SENTENCE2'][i])
    t1 = Tokenizer(df['SENTENCE1'][i])
    t2 = Tokenizer(df['SENTENCE2'][i])

    token_list1 = t1.tokenize()
    token_list2 = t2.tokenize()
    try:
      token_list1 = token_list1.remove(" ")
      token_list2 = token_list2.remove(" ")
    except:
      pass

    #print(token_list1)
    cosine_data.append(cosine_similarity(token_list1,token_list2))
    jaccard_data.append(jaccard_similarity(token_list1,token_list2))
    bigram_data.append(bigram_similarity(token_list1,token_list2))
    syno_data.append(syno_match(token_list1,token_list2))


    token_list1 = ' '.join(' '.join(token_list1).split())
    token_list2 = ' '.join(' '.join(token_list2).split())

    corpus.append([token_list1,token_list2])

  #print(corpus)
  features = pd.DataFrame({"cosine_similarity":cosine_data,
                          "jaccard_similarity":jaccard_data,
                          "bigram_matching":bigram_data,
                          "syno_match":syno_data})
  
  print(features.head())
  print(len(features))
  #print(max(syno_data),min(syno_data))

  return features

In [515]:
feat = features_extraction(df)

   cosine_similarity  jaccard_similarity  bigram_matching  syno_match
0                  2                   1                0           1
1                  1                   1                0           2
2                  2                   2                0           2
3                  2                   2                0           2
4                  2                   2                0           2
6250


#SPLITTING THE DATASET

In [516]:
X_train, X_valid, y_train, y_valid = train_test_split(feat,label, test_size = 0.3, random_state = 100)

#SVM MODEL

In [517]:
svc_model = SVC(C=0.1, gamma=1, kernel='poly') 

In [518]:
svc_model.fit(X_train, y_train) 
prediction = svc_model.predict(X_valid)

In [519]:
cm= confusion_matrix(y_valid, prediction)  
cm

array([[940, 120],
       [472, 343]])

In [520]:
print(accuracy_score(y_valid,prediction))

0.6842666666666667


In [521]:
print(classification_report(y_valid, prediction))

              precision    recall  f1-score   support

           0       0.67      0.89      0.76      1060
           1       0.74      0.42      0.54       815

    accuracy                           0.68      1875
   macro avg       0.70      0.65      0.65      1875
weighted avg       0.70      0.68      0.66      1875



In [546]:
pickle.dump(svc_model, open('svc_model.pkl', 'wb'))

#RANDOM FOREST

In [522]:
rfc_model = rfc(criterion='gini',max_depth=6,max_features='auto',n_estimators=500)

In [523]:
rfc_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=500)

In [524]:
y_pred= rfc_model.predict(X_valid)  

In [525]:
cm= confusion_matrix(y_valid, y_pred)  
cm

array([[939, 121],
       [471, 344]])

In [526]:
print(accuracy_score(y_valid,y_pred))

0.6842666666666667


In [527]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.89      0.76      1060
           1       0.74      0.42      0.54       815

    accuracy                           0.68      1875
   macro avg       0.70      0.65      0.65      1875
weighted avg       0.70      0.68      0.66      1875



In [548]:
pickle.dump(rfc_model, open('rfc_model.pkl', 'wb'))

#TEST DATA

In [531]:
test_df = pd.read_csv("Test.csv")

In [532]:
test_df.head()

,id,SENTENCE1,SENTENCE2,label
0,1,24 फरवरी 1848 को फरवरी की क्रांति के बाद जब अन...,"इसका गठन तब हुआ था जब 24 फरवरी, 1848 को फरवरी ...",0
1,2,लॉस एंजिल्स में जिम अपने सबसे अच्छे दोस्त माइक...,माइक अलोंजो लॉस एंजिल्स में अपने सबसे अच्छे दो...,0
2,3,"टर्नरी समूह के सिद्धांत में, विस्तारित टर्नरी ...","परिमित समूह सिद्धांत में, विस्तारित टर्नरी गोल...",0
3,4,लेखक अपनी वेबसाइट पर और अपने संस्थागत भंडार मे...,लेखक अपनी वेबसाइट पर और अपने संस्थागत भंडार मे...,1
4,5,कॉलेज के प्रमुख स्टीव मार्शल हैं और सहायक प्रम...,कॉलेज के प्रमुख डैनी हबबॉल हैं और सहायक स्टीव ...,0


In [533]:
len(test_df)

7806

In [534]:
test_df = test_df.sample(frac=0.4)

In [535]:
test_df = test_df.reset_index(drop=True)

In [536]:
feat = features_extraction(test_df)
lab = test_df["label"]

   cosine_similarity  jaccard_similarity  bigram_matching  syno_match
0                  1                   1                0           1
1                  2                   2                0           2
2                  1                   1                0           1
3                  2                   2                0           2
4                  2                   2                1           2
3122


#SVM ON TEST DATA

In [537]:
prediction = svc_model.predict(feat)

In [538]:
cm= confusion_matrix(lab, prediction)
cm

array([[1556,  163],
       [ 826,  577]])

In [539]:
print(accuracy_score(lab,prediction))

0.6832158872517617


In [540]:
print(classification_report(lab, prediction))

              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1719
           1       0.78      0.41      0.54      1403

    accuracy                           0.68      3122
   macro avg       0.72      0.66      0.65      3122
weighted avg       0.71      0.68      0.66      3122



#RFC ON TEST DATA

In [541]:
y_pred= rfc_model.predict(feat) 

In [542]:
cm= confusion_matrix(lab, prediction)
cm

array([[1556,  163],
       [ 826,  577]])

In [543]:
print(accuracy_score(lab,prediction))

0.6832158872517617


In [544]:
print(classification_report(lab, prediction))

              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1719
           1       0.78      0.41      0.54      1403

    accuracy                           0.68      3122
   macro avg       0.72      0.66      0.65      3122
weighted avg       0.71      0.68      0.66      3122

